In [1]:
%pip install numpy transformers pandas torch scikit-learn pyarrow accelerate transformers[torch] transformers[sentencepiece] ipywidgets tqdm datetime imblearn sentencepiece papermill

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import time
from datetime import datetime
import string
import random
import logging
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification, AlbertTokenizer, AlbertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Main Code


In [3]:
# Default parameters, Papermill will overwrite these
categories = 'science'
train_data = None
test_data = None
select_model = 0
freeze_layers_up_to = 0
epochs = 1
weight_for_class_0 = None
weight_for_class_1 = None
learning_rate = 0.00001
min_acc = 0.85

In [4]:
# Parameters
freeze_layers_up_to = 0
epochs = 1
weight_for_class_0 = "auto"
weight_for_class_1 = "auto"
learning_rate = 1e-05
min_acc = 0.5
select_model = 5
categories = "health"
train_data = None
test_data = None


In [5]:
def generate_random_string(length=10):
    letters = string.ascii_letters + string.digits
    return ''.join(random.choice(letters) for _ in range(length))

run_id = generate_random_string()

In [6]:
log_dir_mapping = {
  'crime': './results/crime',
  'science': './results/science',
  'health': './results/health',
  'politics': './results/politics',
  'social_media': './results/social_media'
}

log_dir = log_dir_mapping.get(categories, './results')
os.makedirs(log_dir, exist_ok=True)

log_filename = f'training_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
log_filepath = os.path.join(log_dir, log_filename)

logging.basicConfig(filename=log_filepath, filemode='w', format='%(asctime)s - %(message)s', level=logging.INFO)
logging.info(f'\nStarting detection model - {run_id}')

In [7]:
root_dir = '../data'

dataset_mapping = {
  'fa-kes': '../data/crime/FA-KES-Dataset.feather',
  'snope': '../data/crime/snope.feather',
  'covid_claims': '../data/health/covid_claims.feather',
  'covid_fake_news': '../data/health/covid_fake_news_dataset.feather',
  'covid_FNIR': '../data/health/covid_FNIR.feather',
  'fake_news': '../data/politics/fake_news_dataset.feather',
  'isot_dataset': '../data/politics/isot_dataset.feather',
  'liar_dataset': '../data/politics/liar_dataset.feather',
  'pheme': '../data/politics/pheme.feather',
  'politifact': '../data/politics/politifact_dataset.feather',
  'climate': '../data/science/climate_dataset.feather',
  'gossipcop': '../data/social_media/gossipcop.feather',
  'isot_social': '../data/social_media/isot_dataset.feather',
  'isot_multipurpose': '../data/isot_multipurpose_small.feather'
}

def load_data_from_category(category):
    files = os.listdir(os.path.join(root_dir, category))
    dataframes = []
    for file in files:
        if file.endswith('.feather'):
            df = pd.read_feather(os.path.join(root_dir, category, file))
            dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)

def load_as_train_or_test(dataset_list):
    dataframes = []
    for dataset in dataset_list:
        df = pd.read_feather(dataset_mapping.get(dataset))
        dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)

# Use this one when running multiple categories
#combined_dataframes = [load_data_from_category(category) for category in categories] 
# This one is for single category
if train_data is None and test_data is None:
    train_data_set = False
    combined_dataframes = [load_data_from_category(categories)]
    combined_df = pd.concat(combined_dataframes, ignore_index=True)
    combined_df.dropna(inplace=True)
    datasets_size = len(combined_df)
    print("combined dataframe: ")
    print(combined_df)

# This one is when specific training and testing sets are used
if train_data != None and test_data != None:
    train_data_set = True
    train_dataset_name, test_dataset_name = train_data, test_data
    train_df = load_as_train_or_test(train_data)
    test_df = load_as_train_or_test(test_data)
    train_df.dropna(inplace=True)
    test_df.dropna(inplace=True)
    train_dataset_size, test_dataset_size = len(train_df), len(test_df)
    print("training dataframe: ")
    print(train_df.head())
    print("testing dataframe: ")
    print(test_df.head())

combined dataframe: 
                                                    text  label  \
0      Facebook has shuttered a popular group for Mic...      1   
1      We can now officially put to rest all comparis...      0   
2      The Ministry of Health in Bermuda has confirme...      0   
3      Purdue University President Mitch Daniels, the...      1   
4      Locking down much of the country may have help...      1   
...                                                  ...    ...   
13523  in early an outbreak of new coronavirus in chi...      0   
13524  organizers of operation gridlock the first sig...      0   
13525  on may as federal leaders debated how to respo...      0   
13526  in march message started to circulate on faceb...      0   
13527  on feb the beaverton published an article stat...      0   

                                                   title  \
0      FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...   
1       Other Viewpoints: COVID-19 is worse than the f

In [8]:
'''
def load_data_from_category(category, filenames):
    dataframes = []
    for file_name in filenames:
        file_path = os.path.join(root_dir, category, file_name)
        if os.path.exists(file_path):
            df = pd.read_feather(file_path)
            dataframes.append(df)
        else:
            print(f"The file '{file_name}' in the '{category}' category does not exist.")
    return pd.concat(dataframes, ignore_index=True)

root_dir = '../data'
category = 'health'

# List of filenames to include in combined_df
#included_filenames = ['isot_dataset.feather', 'fake_news_dataset.feather', 'pheme.feather', 'liar_dataset.feather', 'politifact_dataset.feather']
included_filenames = ['covid_claims.feather', 'covid_fake_news_dataset.feather', 'covid_FNIR.feather']
combined_df = load_data_from_category(category, included_filenames)

# Drop NaN values
combined_df.dropna(inplace=True)

# Print the resulting DataFrame
print(combined_df)
'''

'\ndef load_data_from_category(category, filenames):\n    dataframes = []\n    for file_name in filenames:\n        file_path = os.path.join(root_dir, category, file_name)\n        if os.path.exists(file_path):\n            df = pd.read_feather(file_path)\n            dataframes.append(df)\n        else:\n            print(f"The file \'{file_name}\' in the \'{category}\' category does not exist.")\n    return pd.concat(dataframes, ignore_index=True)\n\nroot_dir = \'../data\'\ncategory = \'health\'\n\n# List of filenames to include in combined_df\n#included_filenames = [\'isot_dataset.feather\', \'fake_news_dataset.feather\', \'pheme.feather\', \'liar_dataset.feather\', \'politifact_dataset.feather\']\nincluded_filenames = [\'covid_claims.feather\', \'covid_fake_news_dataset.feather\', \'covid_FNIR.feather\']\ncombined_df = load_data_from_category(category, included_filenames)\n\n# Drop NaN values\ncombined_df.dropna(inplace=True)\n\n# Print the resulting DataFrame\nprint(combined_df)\n

In [9]:
# May need to include more columns to process metadata
if train_data is None and test_data is None:
  texts = combined_df['text'].values
  labels = combined_df['label'].values
  num_classes = combined_df['label'].nunique()
  print(texts[:5])
  print(labels[:5])

  unique_classes, class_counts = np.unique(labels, return_counts=True)

if train_data != None and test_data != None:
  train_texts = train_df['text'].values
  train_labels = train_df['label'].values
  test_texts = test_df['text'].values
  test_labels = test_df['label'].values
  num_classes = max(train_df['label'].nunique(), test_df['label'].nunique())
  print(train_texts[:5])
  print(train_labels[:5])
  print(test_texts[:5])
  print(test_labels[:5])

  unique_classes, class_counts = np.unique(train_labels, return_counts=True)

for class_label, count in zip(unique_classes, class_counts):
  print(f"Class {class_label}: {count} instances")

print(f"Number of unique classes: {num_classes}")

['Facebook has shuttered a popular group for Michiganders who oppose their governor’s extreme lockdown measures, fueling debate about free speech during the coronavirus crisis.\n\n‘Michiganders Against Excessive Quarantine’ had more than 380,000 members, with media reports describing it as one of the largest anti-lockdown groups in the nation. It’s not clear if Facebook has provided an explanation for the group’s removal.\n\nVisiting the page gives the following message: “Sorry, this content isn’t available right now.”\n\nGarrett Soldano of Mattawan, Michigan created Michiganders Against Excessive Quarantine in April after becoming disenchanted with Governor Gretchen Whitmer’s stay-at-home order – a policy which he initially supported.\n\nThe group made headlines around the world after organizing ‘Operation Gridlock’, a protest in which thousands of cars created bumper-to-bumper traffic in the capital, Lansing.\n\nProtesters were encouraged to stay in their automobiles and adhere to so

In [10]:
# Split the dataset into training and testing sets
if train_data is None and test_data is None:
  train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=7623)
  print("Split category data into training and testing sets")

Split category data into training and testing sets


In [11]:
balancing_strategy = None

if weight_for_class_0 is None and weight_for_class_1 is None:
  balancing_strategy = 'Default weights'
  # Calculate class frequencies
  class_counts = np.bincount(labels) if train_data is None else np.bincount(train_labels)
  total_samples = np.sum(class_counts)

  # Calculate class weights, the original distribution of weights
  weight_for_class_0 = class_counts[0] / sum(class_counts)
  weight_for_class_1 = class_counts[1] / sum(class_counts)
elif weight_for_class_0 == 'auto' and weight_for_class_1 == 'auto':
  balancing_strategy = 'Auto weights'
  # Calculate class frequencies
  class_counts = np.bincount(labels) if train_data is None else np.bincount(train_labels)
  total_samples = np.sum(class_counts)

  # Calculate class weights
  weight_for_class_0 = class_counts[1] / sum(class_counts)
  weight_for_class_1 = class_counts[0] / sum(class_counts)
else:
  balancing_strategy = 'Manual weights'

print(f"Weight for class 0: {weight_for_class_0:.2f}")
print(f"Weight for class 1: {weight_for_class_1:.2f}")


Weight for class 0: 0.45
Weight for class 1: 0.55


In [12]:
# Load the pre-trained BERT (or BERT variation) model and tokenizer
model_mapping = {
    0: (BertTokenizer, BertForSequenceClassification, 'bert-base-uncased'),
    1: (BertTokenizer, BertForSequenceClassification, 'bert-base-cased'),
    2: (DistilBertTokenizer, DistilBertForSequenceClassification, 'distilbert-base-uncased-finetuned-sst-2-english'),
    3: (DistilBertTokenizer, DistilBertForSequenceClassification, 'distilbert-base-uncased'),
    4: (RobertaTokenizer, RobertaForSequenceClassification, 'roberta-base'),
    5: (AlbertTokenizer, AlbertForSequenceClassification, 'albert-base-v2')
}

tokenizer_class, model_class, model_name = model_mapping.get(select_model, (None, None, None))

if tokenizer_class and model_class and model_name:
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = model_class.from_pretrained(model_name, num_labels=2)
else:
    logging.error(f"Invalid model selection: {select_model}")

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Freeze the layers up to the specified layer
if freeze_layers_up_to > 0:
    for param in model.parameters():
        param.requires_grad = False

    if select_model == 0 or select_model == 1:
        print("Layers: "+str(len(model.bert.encoder.layer)))
        for param in model.bert.embeddings.parameters():
            param.requires_grad = True
    elif select_model == 2 or select_model == 3:
        print("Layers: "+str(len(model.distilbert.transformer.layer)))
        for param in model.distilbert.embeddings.parameters():
            param.requires_grad = True
    elif select_model == 4:
        print("Layers: "+str(len(model.roberta.encoder.layer)))
        for param in model.roberta.embeddings.parameters():
            param.requires_grad = True
    elif select_model == 5:
        print("Layers: "+str(len(model.albert.encoder.albert_layer_groups)))
        for param in model.albert.embeddings.parameters():
            param.requires_grad = True


In [14]:
# Tokenize the texts
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [15]:
# Convert the encodings to PyTorch tensors
train_inputs = torch.tensor(train_encodings['input_ids'])
train_masks = torch.tensor(train_encodings['attention_mask'])
train_labels = torch.tensor(train_labels)

test_inputs = torch.tensor(test_encodings['input_ids'])
test_masks = torch.tensor(test_encodings['attention_mask'])
test_labels = torch.tensor(test_labels)

# Create a DataLoader for training and testing
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_dataloader = DataLoader(test_data, batch_size=8)

In [16]:
# Check for GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [17]:
# Move the model and data to the GPU
model.to(device)
train_inputs, train_masks, train_labels = train_inputs.to(device), train_masks.to(device), train_labels.to(device)
test_inputs, test_masks, test_labels = test_inputs.to(device), test_masks.to(device), test_labels.to(device)

# Define class weights based on class imbalance
class_weights = torch.tensor([weight_for_class_0, weight_for_class_1], dtype=torch.float)

# Define the loss function with class weights
loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).to(device))

# Define the optimizer with a learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Model training code
def train_model(model, epochs, optimizer, loss_fn, train_dataloader):
    for epoch in range(epochs):
        model.train()
        model.to(device)
        total_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()
            inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
            outputs = model(**inputs)
            loss = loss_fn(outputs.logits, inputs['labels'])
            loss.backward()
            optimizer.step()
        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch+1} average train loss: {avg_train_loss}")
        logging.info(f"Epoch {epoch+1} average train loss: {avg_train_loss}")

# Fine-tune the pre-trained BERT model
train_start_time = time.time()
train_model(model, epochs, optimizer, loss_fn, train_dataloader)
train_end_time = time.time()

/tmp/ipykernel_22229/2533579605.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(class_weights).to(device))


Epoch 1 average train loss: 0.0


In [18]:
# Evaluate the model
eval_start_time = time.time()
model.eval()
predictions = []
for batch in test_dataloader:
    inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': None}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions.extend(logits.argmax(dim=1).cpu().tolist())
eval_end_time = time.time()

In [19]:
# Calculate additional metrics
precision = precision_score(test_labels.cpu(), predictions)
recall = recall_score(test_labels.cpu(), predictions)
f1 = f1_score(test_labels.cpu(), predictions)
accuracy = accuracy_score(test_labels.cpu(), predictions)
g_mean = (recall*accuracy)**0.5

In [20]:
save_dir = './models'
os.makedirs(save_dir, exist_ok=True)

saved_model_name = f"bert_model_{run_id}.pt"
model_path = os.path.join(save_dir, saved_model_name)

# Only save model if accuracy meets minimum threshold
if accuracy > min_acc:
    torch.save(model.state_dict(), model_path)

In [21]:
# Log the results
logging.info("Evaluation Results")
logging.info(f"Training time: {train_end_time - train_start_time} seconds")
logging.info(f"Inference time: {eval_end_time - eval_start_time} seconds")
logging.info(f"Precision: {precision}")
logging.info(f"Recall: {recall}")
logging.info(f"F-score: {f1}")
logging.info(f"Accuracy: {accuracy}")
logging.info(f"G-mean: {g_mean}")
logging.info("Additional Info")
logging.info(f"Model name: {model_name}")
if not train_data_set:
  logging.info(f"Datasets list: {categories}")
  logging.info(f"Dataset size: {datasets_size}")
else:
  logging.info(f"Datasets list: {categories} Train: {train_dataset_name}, Test: {test_dataset_name}")
  logging.info(f"Dataset size: Train: {train_dataset_size}, Test: {test_dataset_size}")
logging.info(f"Layers frozen: {freeze_layers_up_to}")
logging.info(f"Learning rate: {learning_rate}")
logging.info(f"Class weights: {balancing_strategy} - {[weight_for_class_0, weight_for_class_1]}")
if accuracy > min_acc:
  logging.info(f"Model saved to: {model_path}")
else:
  logging.info("Model not saved, didn't meet minimum accuracy threshold")

# Print the evaluation results
print("Evaluation Results:")
print(f"Training time: {train_end_time - train_start_time} seconds")
print(f"Inference time: {eval_end_time - eval_start_time} seconds")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F-score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"G-mean: {g_mean}")

Evaluation Results:
Training time: 230.77180671691895 seconds
Inference time: 20.922882080078125 seconds
Precision: 0.8943820224719101
Recall: 0.7467166979362101
F-score: 0.8139059304703475
Accuracy: 0.8466722830665543
G-mean: 0.7951253558059694
